In [60]:
import pandas as pd
import streamlit as st
import pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
import scipy.sparse
import warnings
warnings.filterwarnings('ignore')

In [62]:
# df = pickle.load(open('Naukri_job.pkl','rb'))
# df

In [66]:
# streamlit_job_search.py

# Load dataset
st.title("Model Deployment using Naukri CSV")

# st.write("Upload your CSV and deploy a basic ML model instantly!")


# Load your job dataset
df = pd.read_csv('naukri.csv')  # <-- Update with your file

# Show dataset
st.subheader("Dataset Preview")
st.dataframe(df.head())

# Choose target
st.subheader("Select Target Column 🎯")
target = st.selectbox("Target Column", df.columns)


# Preprocess
X = df.drop(columns=[target])
y = df[target]

# Handle categorical features
cat_features = X.select_dtypes(include=['object']).columns.tolist()

for col in cat_features:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col].astype(str))

# Handle missing values
X.fillna(0, inplace=True)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.77, random_state=5)


# Check if classification or regression
if y.dtype == 'object' or len(y.unique()) <= 20:
    model = RandomForestClassifier()
    problem_type = "Classification"
else:
    model = RandomForestRegressor()
    problem_type = "Regression"

# Train model
model.fit(X_train, y_train)

# Model trained
st.success(f"{problem_type} model trained successfully!")

# Input for prediction
st.subheader("Make a Prediction 🎯")


input_data = {}
for col in X.columns:
    if col in cat_features:
        unique_vals = df[col].unique().tolist()
        input_val = st.selectbox(f"Select {col}", unique_vals)
        le = LabelEncoder()
        le.fit(df[col].astype(str))
        input_val = le.transform([input_val])[0]
    else:
        input_val = st.number_input(f"Input {col}", value=float(X[col].mean()))
    input_data[col] = input_val

# Predict button
if st.button("Predict"):
    input_df = pd.DataFrame([input_data])
    prediction = model.predict(input_df)[0]
    st.success(f"Prediction: {prediction}")

